In [4]:
%config IPCompleter.greedy=True
import os
import glob
import json
import MeCab
import pickle
import sys
from collections import Counter
import gzip

# リスト化して保存 
## (pklに保存するにしてもメモリ不足なので没)

In [26]:
def step1():
  for path in glob.glob("../wakachi/contents*"):
    objs = []
    for name in glob.glob("{}/*".format(path)):
      #print(name)
      obj = json.load(gzip.open(name,"rt"))
      #リスト化
      obj['bodies'] = obj['bodies'].split()
      #print(obj)
      objs.append(obj)
      #sys.exit()
    save =path.replace("../wakachi/contents","")
    with open('./pkl/objs{}.pkl'.format(save), 'wb') as f:
      f.write( pickle.dumps(objs) )

step1()

../wakachi/contents1250_1349/12807323.gz


SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 単語にインデックス付与

In [22]:
def step2(): # term index
  term_index = {}  
  for path in glob.glob("../wakachi/contents*"):
    for name in glob.glob("{}/*".format(path)):
      print(path,name)
      obj = json.load(gzip.open(name,"rt"))
      obj['bodies'] = obj['bodies'].split()
      for term in obj['bodies']:
        if term_index.get(term) is None:
          term_index[term] = len(term_index)
      #print(term_index)  
  open('term/term_index.json', 'w').write( json.dumps(term_index, indent=2, ensure_ascii=False) )

step2()

./pkl/objs1250_1349.pkl


SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# SVM形式に変換

In [47]:
# make svm format
def  step3():
  term_index = json.load( open('term/term_index.json') )
  for path in glob.glob("../wakachi/contents*"):
    #print(path)
    for name in glob.glob("{}/*".format(path)):
      obj = json.load(gzip.open(name,"rt"))
      obj['bodies'] = obj['bodies'].split()
      
      #１つのデータごとにSVM形式
      if "国内の事件・事故" in obj["tag"] or "海外の事件・事故" in obj["tag"]:
        state = 1.0
      else:
        state = 0.0

      # 各単語の出現回数を数える（スペース区切りで数えてくれる）
      term_freq = dict(Counter(obj['bodies']))
    
      #番号と出現回数　、番号は0始まりから1始まりに変換　　　出現回数は形態素の出現頻度を用いる
      freqs = len(obj['bodies'])
      context = [ [term_index[term]+1,freq/freqs] for term, freq in term_freq.items() ]    
      #print(context)

      #liblinearに合わせるため、番号順に並べる
      context = sorted(context, key=lambda x:x[0])
      #print(context)
    
      #liblinearに合わせるため、 リストからスペース区切りに変換
      context = ' '.join( ['%d:%f'%(term, freq) for term, freq in context] )
    
      # 正解ラベルと特徴量の形に
      print( state, context )

step3()

0.0 1:0.010381 2:0.003460 3:0.006920 4:0.003460 5:0.006920 6:0.003460 7:0.003460 8:0.013841 9:0.006920 10:0.003460 11:0.006920 12:0.038062 13:0.003460 14:0.003460 15:0.010381 16:0.003460 17:0.010381 18:0.010381 19:0.041522 20:0.027682 21:0.003460 22:0.003460 23:0.003460 24:0.034602 25:0.003460 26:0.027682 27:0.003460 28:0.010381 29:0.044983 30:0.031142 31:0.003460 32:0.003460 33:0.027682 34:0.003460 35:0.003460 36:0.003460 37:0.003460 38:0.013841 39:0.013841 40:0.003460 41:0.003460 42:0.003460 43:0.003460 44:0.003460 45:0.003460 46:0.003460 47:0.003460 48:0.003460 49:0.003460 50:0.003460 51:0.003460 52:0.010381 53:0.003460 54:0.003460 55:0.003460 56:0.003460 57:0.003460 58:0.003460 59:0.003460 60:0.003460 61:0.010381 62:0.006920 63:0.003460 64:0.006920 65:0.006920 66:0.006920 67:0.003460 68:0.006920 69:0.010381 70:0.020761 71:0.003460 72:0.038062 73:0.003460 74:0.003460 75:0.003460 76:0.003460 77:0.003460 78:0.003460 79:0.003460 80:0.003460 81:0.003460 82:0.003460 83:0.003460 84:0.0034